# Make parameter submission scripts for HPC 

## Mitophagy rate sweep

Author: Juvid Aryaman

In [1]:
import numpy as np
import pandas as pd
import mitonetworks.det as mtd
import mitonetworks.stoch as mts

In [2]:
import warnings
warnings.simplefilter("ignore",RuntimeWarning)

In [3]:
nominal_params = {
    'gamma':0.03785142857142858,
	'beta':33.12,
	'kappa':11.662903457629223,
	'b':1.2416523075924095e-05,
	'mu':0.023,
	'xi':0.0,
	'delta':1.0
}

In [4]:
c_file_param_ordering_convention = ['xi','beta','gamma','kappa','b','mu','delta']

In [5]:
hpc_workdir = '$WORK/networks/sing_birth_fus/mu_sw'

In [6]:
c_filename = 'closed_loop_deg_single'

In [7]:
hpc_jobname = 'mu_sw'

In [8]:
ms = mts.SubmitHPCSoluableFeedbackControl(ss_definition=mtd.E_linear_feedback_control_ss,
                                 nominal_params=nominal_params,hpc_workdir=hpc_workdir,
                                c_filename=c_filename, c_file_param_ordering_convention=c_file_param_ordering_convention,
                                hpc_jobname=hpc_jobname)

In [9]:
ics, h_opt, err = ms.find_ss_h_target(nominal_params)
print((ics[0]+[2])/float(np.sum(ics)))

[0.32832833]


Find optimal $\kappa,\mu$ pairs for $h=0.3$ and $n=1000$

In [10]:
n_target = 1000.0
kappa_space = np.linspace(-1000,500,2001)
mu_sp = [0.75,1.5,1.75,2]

opt_params = []

params = nominal_params.copy()

for mu_val in mu_sp:
    params['mu'] = mu_val*nominal_params['mu']
    
    n_sp = []
    n_ind_sp = []
    fs_sp = []
    ics_sp = []
    
    # Find corresponding kappa to keep copy number constant
    for j, kappa_val in enumerate(kappa_space): 
        params['kappa'] = kappa_val
        ics, h_opt, err = ms.find_ss_h_target(params)
        
        if err == 1:
            n_sp.append(np.sum(ics))
            n_ind_sp.append(j)
            fs_sp.append((ics[0]+[2])/float(np.sum(ics)))
            ics_sp.append(ics)
        else:
            continue
    n_sp = np.array(n_sp)
    n_ind_opt =  np.argmin(np.abs(n_sp - n_target))
    
    print(fs_sp[n_ind_opt]) # fs at this optimal parametrization. This should be invariant
    
    if abs(n_sp[n_ind_opt] - n_target) > 1.5:
        raise Exception("Could not find a kappa for this mu where n is close to n_target")
    
    opt_params.append([params['mu'],kappa_space[n_ind_sp[n_ind_opt]],ics_sp[n_ind_opt]])
    

[0.32832833]
[0.32832833]
[0.328]
[0.328]


In [11]:
opt_params

[[0.01725, 258.5, array([326, 373, 140, 160])],
 [0.0345, -483.25, array([326, 373, 140, 160])],
 [0.04025, -730.0, array([326, 374, 140, 160])],
 [0.046, -977.5, array([326, 374, 140, 160])]]

Make submission

In [15]:
master_script = open(ms.out_dir+"/master.sh","w")
# Add the .c compilation line to the master script
master_script.write('gcc -o3 {0}.c -lm -o {0}.ce\n'.format(ms.c_filename))

# Output dataframe with parametriations
param_df =  []

for i, p in enumerate(opt_params):
    params['mu'] = p[0]
    params['kappa'] = p[1]
    param_dict = params.copy()
    
    ics, h_opt, err = ms.find_ss_h_target(params)
    
    for k, param in enumerate(ms.param_convention): param_dict[param+'_init'] = ics[k]
    param_df.append(pd.DataFrame(param_dict, index=[0]))
    
    ics, h_opt, err = ms.find_ss_h_target(params)
    if err == 1:
        ms.make_submission_script(i,ics,params)
        # Add the submission to a master script
        master_script.write('echo "{}"\n'.format(i))
        master_script.write('mkdir -p {0}/p{1}\n'.format(ms.hpc_workdir,i))
        master_script.write("qsub submission_{}.pbs\n".format(i))
        master_script.write('sleep 0.5\n')
    else:
        raise Exception('SS not found!')
master_script.close()
param_df = pd.concat(param_df, ignore_index=True)
param_df.to_csv(ms.out_dir+'/param_sweep_vals.csv',index=False)    

In [16]:
param_df

,b,beta,delta,gamma,kappa,mf_init,ms_init,mu,wf_init,ws_init,xi
0,0.000012,33.12,1.0,0.037851,258.50,160,140,0.01725,373,326,0.0
1,0.000012,33.12,1.0,0.037851,-483.25,160,140,0.03450,373,326,0.0
2,0.000012,33.12,1.0,0.037851,-730.00,160,140,0.04025,374,326,0.0
3,0.000012,33.12,1.0,0.037851,-977.50,160,140,0.04600,374,326,0.0


In [17]:
param_df['mu']/0.023

0    0.75
1    1.50
2    1.75
3    2.00
Name: mu, dtype: float64